In [1]:
from easyF1.adapters import LivetimingF1adapters, livetimingF1_request, livetimingF1_getdata
from easyF1.season import Season
from easyF1.functions import *
from easyF1.events import *
import urllib
import json
import numpy as np

In [2]:
endpoint = "https://livetiming.formula1.com/static/2024/2024-09-01_Italian_Grand_Prix/2024-09-01_Race/CarData.z.jsonStream"
data = livetimingF1_getdata(endpoint, stream=True)
data

{'00:01:13.215': '7ZS9DsIwDITfxXNbJXb+mrXiDWABMVSoEkioQ+lW9d1pw85wCwxeLlGSk+yL9S10GOfpMbwoXxY6zTfKxIZdbdra2KPlbFL2oQlepE1nqqjrp+3xQnaX7t6P4/AsB4ayqYiLSlFX1H/2+7Ku5QryOdBnDWpEO7RwqQE1JtDIaDjMqBH9f0ZT5YhOKjoADp5xtEfvQWNAS41oqglLdbN+41NqRQIroBRQCigF1N8BKjZWRDh6BZQCSgGlgPoJoK7rGw==',
 '00:01:14.375': '7daxDoIwEAbgd7kZyPXag9KV+Aa6aByIIdHEMCgb4d2Fujv8iwy3HKTwJ+1x+cJMh3F6PYY3pctMp+lGiYQllNyW7I5OEsekTRWcl6j+TAV1/Wt9eya3le7ej+PwzAtMiQuSXH2uIVf93m+XZcmPoFwAc47RIHpCB2+1RoMRDAraHBE0iH5/QbsqDTqp6AAEeMbRM6qCwRrdaoN2NWJdXaO/gKrVS2ijAWVAGVAG1P6AWv+eZF03oAwoA8qA2htQsWL1ruXagDKgDCgDan9ASeudBjWgDCgDyoD6C1DX5QM=',
 '00:01:15.215': '7ZSxDsIwDET/xXNb2U7Splkr/gAWEEOFKoGEOpRuVf+dNuwMt8Dg5RIlOcm+WG+hwzhPj+FF6bLQab5RImX1Jbcly1E0cUwhVnVwwnKmgrp+2h4vJLt0934ch2c+YEpckGZ1WX3W8Nnvy7rmK8jnQZ8wakQ7FLjUGjVG0KhoOKqoEf1/RVPVBp1UdAA8PONojyGAxhottUFTjViqm/Ubn6I4bppggDJAGaAMUP8GqLYS59ixAcoAZYAyQP0GUNf1DQ==',
 '00:01:16.054': '7ZS9DsIwDITfxXOLHJP/teINYAExVKgSSChD6Rb13ZuGneEWliyXKMkn2RfrMp3SMr+mD8VbpsvyoEjConsOPauzksg+mnAwToL

In [15]:


from easyF1.utils.helper import *
text = list(data.values())[1000]
json.loads(zlib.decompress(base64.b64decode(text), -zlib.MAX_WBITS).decode('utf-8-sig'))

{'Entries': [{'Utc': '2024-09-01T12:25:41.2561225Z',
   'Cars': {'1': {'Channels': {'0': 8055,
      '2': 97,
      '3': 2,
      '4': 0,
      '5': 100,
      '45': 1}},
    '3': {'Channels': {'0': 9475,
      '2': 151,
      '3': 4,
      '4': 0,
      '5': 100,
      '45': 1}},
    '4': {'Channels': {'0': 4127, '2': 33, '3': 1, '4': 0, '5': 0, '45': 0}},
    '10': {'Channels': {'0': 11031, '2': 0, '3': 1, '4': 24, '5': 0, '45': 0}},
    '11': {'Channels': {'0': 3364, '2': 33, '3': 0, '4': 0, '5': 0, '45': 0}},
    '14': {'Channels': {'0': 10487,
      '2': 151,
      '3': 3,
      '4': 73,
      '5': 0,
      '45': 0}},
    '16': {'Channels': {'0': 9446, '2': 226, '3': 7, '4': 0, '5': 0, '45': 1}},
    '18': {'Channels': {'0': 4227, '2': 21, '3': 1, '4': 0, '5': 0, '45': 0}},
    '20': {'Channels': {'0': 11810,
      '2': 257,
      '3': 6,
      '4': 100,
      '5': 0,
      '45': 0}},
    '22': {'Channels': {'0': 7449, '2': 79, '3': 2, '4': 51, '5': 0, '45': 1}},
    '23': {'Chann

In [9]:
def parse(text: str, zipped: bool = False) -> Union[str, dict]:
    """
    FastF1 code
    """
    if text[0] == '{':
        return json.loads(text)
    if text[0] == '"':
        text = text.strip('"')
    if zipped:
        text = zlib.decompress(base64.b64decode(text), -zlib.MAX_WBITS)
        return parse(text.decode('utf-8-sig'))
    return text

def parse_hash(hash_code):
    tl=12
    return parse(hash_code, zipped=True)

def parse_helper_for_nested_dict(info, record, prefix=""):
    for info_k, info_v in info.items():
        if isinstance(info_v, list): record = {**record, **{**{info_k + "_" + str(sector_no+1) + "_" + k : v  for sector_no in range(len(info_v)) for k,v in info_v[sector_no].items()}}}
        elif isinstance(info_v, dict): record = parse_helper(info_v, record, prefix= prefix + info_k + "_")
        else: record = {**record, **{prefix + info_k : info_v}}
    return record


records = []
for ts, v in data.items():

    parsed_entry = parse(v, zipped=True)
    for entry in parsed_entry["Entries"]:
        utc = entry["Utc"]
        for driver_entry in entry["Cars"].items():
            
            record = {
                **{
                    "timestamp" : ts,
                    "Utc" : utc,
                    "DriverNo" : driver_entry[0]
                },
                **driver_entry[1]["Channels"]
                }
            
            records.append(record)

In [20]:
pd.DataFrame(records)

,timestamp,Utc,DriverNo,0,2,3,4,5,45
0,00:01:13.215,2024-09-01T12:08:56.653398Z,1,0,0,0,0,0,0
1,00:01:13.215,2024-09-01T12:08:56.653398Z,3,0,0,0,0,0,0
2,00:01:13.215,2024-09-01T12:08:56.653398Z,4,0,0,0,0,0,0
3,00:01:13.215,2024-09-01T12:08:56.653398Z,10,0,0,0,0,0,0
4,00:01:13.215,2024-09-01T12:08:56.653398Z,11,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
584355,02:12:50.518,2024-09-01T14:20:34.4365414Z,44,5798,92,4,7,0,0
584356,02:12:50.518,2024-09-01T14:20:34.4365414Z,55,8606,100,2,25,0,0
584357,02:12:50.518,2024-09-01T14:20:34.4365414Z,63,7804,108,3,14,0,0
584358,02:12:50.518,2024-09-01T14:20:34.4365414Z,77,6878,93,3,12,0,0


In [104]:
pd.DataFrame(records)

,SessionKey,timestamp,DriverNo,Meeting_Key,Meeting_Name,Meeting_OfficialName,Meeting_Location,Meeting_Country_Key,Meeting_Country_Code,Meeting_Country_Name,Meeting_Circuit_Key,Meeting_Circuit_ShortName,ArchiveStatus_Status,Key,Type,Name,StartDate,EndDate,GmtOffset,Path
0,0,00:00:00.000,Key,1244,Italian Grand Prix,FORMULA 1 PIRELLI GRAN PREMIO D’ITALIA 2024,Monza,13,ITA,Italy,39,Monza,Generating,9590,Race,Race,2024-09-01T15:00:00,2024-09-01T17:00:00,02:00:00,2024/2024-09-01_Italian_Grand_Prix/2024-09-01_...


In [13]:
records = []

def parse_helper(info, record, prefix=""):
    for info_k, info_v in info.items():
        if isinstance(info_v, list): record = {**record, **{**{info_k + "_" + str(sector_no+1) + "_" + k : v  for sector_no in range(len(info_v)) for k,v in info_v[sector_no].items()}}}
        elif isinstance(info_v, dict): record = parse_helper(info_v, record, prefix= prefix + info_k + "_")
        else: record = {**record, **{prefix + info_k : info_v}}
    return record

for ts, value in data.items():
    if "Withheld" in value.keys(): withTheId = value["Withheld"]
    else: withTheId = None
    
    for driver_no, info in value["Lines"].items():
        record= {
                "SessionKey" : 0,
                "timestamp" : ts,
                "DriverNo" : driver_no
            }

        record = parse_helper(info, record)

        records.append(record)

In [15]:
pd.DataFrame(records).dtypes

SessionKey                        int64
timestamp                        object
DriverNo                         object
Line                            float64
RacingNumber                     object
PersonalBestLapTime_Value        object
BestSectors_1_Value              object
BestSectors_2_Value              object
BestSectors_3_Value              object
BestSpeeds_I1_Value              object
BestSpeeds_I2_Value              object
BestSpeeds_FL_Value              object
BestSpeeds_ST_Value              object
BestSpeeds_ST_Position          float64
BestSpeeds_I1_Position          float64
BestSectors_1_Position          float64
BestSpeeds_I2_Position          float64
BestSpeeds_FL_Position          float64
BestSectors_2_Position          float64
BestSectors_0_Position          float64
BestSectors_0_Value              object
PersonalBestLapTime_Lap         float64
PersonalBestLapTime_Position    float64
dtype: object

In [25]:
records = []
for ts, ts_value in data.items():
    for driver_no, driver_data in ts_value.items():
        if isinstance(driver_data["LapPosition"], list):
            for position in driver_data["LapPosition"]:
                record = {
                        "SessionKey" : 0,
                        "timestamp" : ts,
                        "DriverNo" : driver_no,
                        "Lap" : 0,
                        "LapPosition" : position
                    }
                records.append(record)
            
        
        elif isinstance(driver_data["LapPosition"], dict):
            for lap, position in driver_data["LapPosition"].items():
                record = {
                        "SessionKey" : 0,
                        "timestamp" : ts,
                        "DriverNo" : driver_no,
                        "Lap" : lap,
                        "LapPosition" : position
                    }
                records.append(record)
    
pd.DataFrame(records)

,SessionKey,timestamp,DriverNo,Lap,LapPosition
0,0,00:00:02.591,4,0,1
1,0,00:00:02.591,81,0,2
2,0,00:00:02.591,63,0,3
3,0,00:00:02.591,16,0,4
4,0,00:00:02.591,55,0,5
...,...,...,...,...,...
1023,0,02:11:29.953,20,53,9
1024,0,02:11:39.081,23,53,9
1025,0,02:11:40.232,14,53,11
1026,0,02:11:50.230,3,53,12


In [27]:
def parse_helper(info, record, prefix=""):
    for info_k, info_v in info.items():

        if isinstance(info_v, list):
            record = {
                **record,
                **{
                    **{info_k + "_" + str(sector_no+1) + "_" + k : v  for sector_no in range(len(info_v)) for k,v in info_v[sector_no].items()}
                }
            }

        elif isinstance(info_v, dict):
            record = parse_helper(info_v, record, prefix= prefix + info_k + "_")
            # record = {
            #     **record,
            #     **{
            #         info_k + "_" + k : v for k,v in info_v.items()
            #     }
            # }

        else:
            record = {
                **record,
                **{
                    prefix + info_k : info_v 
                }
            }
    
    return record

records = []
for ts, value in data.items():
    if "Withheld" in value.keys(): withTheId = value["Withheld"]
    else: withTheId = None
    
    for driver_no, info in value["Lines"].items():
        record= {
                "SessionKey" : 0,
                "timestamp" : ts,
                "DriverNo" : driver_no
            }

        record = parse_helper(info, record)

        records.append(record)

df = pd.DataFrame(records)

AttributeError: 'list' object has no attribute 'items'

In [4]:
df

,SessionKey,timestamp,DriverNo,GapToLeader,IntervalToPositionAhead_Value,IntervalToPositionAhead_Catching,Line,Position,ShowPosition,RacingNumber,...,Sectors_2_Segments_7_Status,Sectors_1_PreviousValue,Sectors_2_PreviousValue,NumberOfLaps,Sectors_0_Value,Sectors_0_OverallFastest,Sectors_0_PersonalFastest,Sectors_0_PreviousValue,BestLapTime_Lap,NumberOfPitStops
0,0,00:00:02.591,4,LAP 1,LAP 1,False,1.0,1,True,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,00:00:02.591,81,,,False,2.0,2,True,81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,00:00:02.591,63,,,False,3.0,3,True,63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,00:00:02.591,16,,,False,4.0,4,True,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,00:00:02.591,55,,,False,5.0,5,True,55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46370,0,02:12:50.731,31,1L,NaN,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46371,0,02:12:50.731,22,46L,45L,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46372,0,02:12:50.731,18,1L,NaN,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46373,0,02:12:50.731,77,1L,NaN,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df["BestLapTime_Value"]

0           
1           
2           
3           
4           
        ... 
28918    NaN
28919    NaN
28920    NaN
28921    NaN
28922    NaN
Name: BestLapTime_Value, Length: 28923, dtype: object

In [9]:
data['00:00:02.591']["Lines"]["4"]

{'GapToLeader': '',
 'IntervalToPositionAhead': {'Value': '', 'Catching': False},
 'Line': 1,
 'Position': '1',
 'ShowPosition': True,
 'RacingNumber': '4',
 'Retired': False,
 'InPit': True,
 'PitOut': False,
 'Stopped': False,
 'Status': 80,
 'Sectors': [{'Stopped': False,
   'Value': '',
   'Status': 0,
   'OverallFastest': False,
   'PersonalFastest': False},
  {'Stopped': False,
   'Value': '',
   'Status': 0,
   'OverallFastest': False,
   'PersonalFastest': False},
  {'Stopped': False,
   'Value': '',
   'Status': 0,
   'OverallFastest': False,
   'PersonalFastest': False}],
 'Speeds': {'I1': {'Value': '',
   'Status': 0,
   'OverallFastest': False,
   'PersonalFastest': False},
  'I2': {'Value': '',
   'Status': 0,
   'OverallFastest': False,
   'PersonalFastest': False},
  'FL': {'Value': '',
   'Status': 0,
   'OverallFastest': False,
   'PersonalFastest': False},
  'ST': {'Value': '',
   'Status': 0,
   'OverallFastest': False,
   'PersonalFastest': False}},
 'BestLapTime': 

In [1]:
session_key = 10
records = []
for key, info in data.items():
    record = {
        **{
            "session_key": session_key,
            "timestamp": key,
        },
        **info
    }
    # yield record
    records.append(record)


pd.DataFrame(records)

NameError: name 'livetimingF1_getdata' is not defined

In [20]:
records = []

endpoint = "https://livetiming.formula1.com/static/2020/2020-07-05_Austrian_Grand_Prix/2020-07-05_Race/SessionData.jsonStream"
# endpoint = "https://livetiming.formula1.com/static/2020/2020-07-05_Austrian_Grand_Prix/2020-07-03_Practice_1/SessionData.jsonStream"
data = livetimingF1_getdata(endpoint, stream=True)
data

records = []
list(list(data.values())[0].values())[0]

for key, value in data.items():
    for driver_no, info in value.items():
        try:
            record = {
                **list(info.values())[0]
            }
            records.append(record)
        except:
            pass
        
pd.DataFrame(records)

,Utc,Lap,SessionStatus,TrackStatus
0,2020-07-05T12:40:04.65Z,1.0,NaN,NaN
1,2020-07-05T13:12:47.228Z,NaN,Started,NaN
2,2020-07-05T13:13:58.983Z,2.0,NaN,NaN
3,2020-07-05T13:15:08.42Z,3.0,NaN,NaN
4,2020-07-05T13:16:18.019Z,4.0,NaN,NaN
...,...,...,...,...
83,2020-07-05T14:42:12.361Z,NaN,NaN,AllClear
84,2020-07-05T14:42:34.813Z,71.0,NaN,NaN
85,2020-07-05T14:43:43.214Z,NaN,Finished,NaN
86,2020-07-05T14:47:24.24Z,NaN,Finalised,NaN


In [22]:
seasons = [
    "2020",
    "2021",
    "2022",
    "2023",
    "2024"
]

season = get_season("2024")

for year in seasons[0:2]:
    print(year)
    try:
        season = get_season(year)
        for meeting in season.meetings[0:2]:
            print(meeting.officialname)
            for session in meeting.sessions:
                try:
                    if hasattr(session,"full_path"):
                        print(session.full_path)
                        display(test(session.full_path))
                except Exception as e:
                    print(e)
                    pass
    except:
        pass

2020
FORMULA 1 ROLEX GROSSER PREIS VON ÖSTERREICH 2020
https://livetiming.formula1.com/static/2020/2020-07-05_Austrian_Grand_Prix/2020-07-03_Practice_1/


,Utc,SessionStatus,TrackStatus
0,2020-07-03T09:00:00.119Z,Started,NaN
1,2020-07-03T10:10:44.359Z,NaN,VSCDeployed
2,2020-07-03T10:11:19.058Z,NaN,VSCEnding
3,2020-07-03T10:11:32.033Z,NaN,AllClear
4,2020-07-03T10:14:14.877Z,NaN,Yellow
5,2020-07-03T10:14:25.646Z,NaN,AllClear
6,2020-07-03T10:19:26.39Z,NaN,Yellow
7,2020-07-03T10:19:35.374Z,NaN,AllClear
8,2020-07-03T09:00:00.119Z,Started,NaN


https://livetiming.formula1.com/static/2020/2020-07-05_Austrian_Grand_Prix/2020-07-03_Practice_2/


,Utc,SessionStatus,TrackStatus
0,2020-07-03T13:00:00.101Z,Started,NaN
1,2020-07-03T13:12:17.665Z,NaN,Yellow
2,2020-07-03T13:12:39.554Z,NaN,AllClear
3,2020-07-03T13:13:37.58Z,NaN,Yellow
4,2020-07-03T13:14:01.343Z,NaN,AllClear
5,2020-07-03T13:15:48.965Z,NaN,Yellow
6,2020-07-03T13:16:11.156Z,NaN,AllClear
7,2020-07-03T13:36:17.38Z,NaN,Yellow
8,2020-07-03T13:36:24.592Z,NaN,AllClear
9,2020-07-03T14:01:08.16Z,NaN,Yellow


https://livetiming.formula1.com/static/2020/2020-07-05_Austrian_Grand_Prix/2020-07-04_Practice_3/


...

,Utc,SessionStatus,TrackStatus
0,2021-04-17T09:00:00.11Z,Started,NaN
1,2021-04-17T09:25:24.312Z,NaN,Yellow
2,2021-04-17T09:25:44.643Z,Aborted,NaN
3,2021-04-17T09:25:44.893Z,NaN,Red
4,2021-04-17T09:28:01.926Z,Inactive,NaN
5,2021-04-17T09:28:02.08Z,NaN,AllClear
6,2021-04-17T09:29:00.138Z,Started,NaN
7,2021-04-17T09:48:58.79Z,NaN,Yellow
8,2021-04-17T09:49:03.246Z,NaN,AllClear
9,2021-04-17T09:55:21.731Z,NaN,Yellow


https://livetiming.formula1.com/static/2021/2021-04-18_Emilia_Romagna_Grand_Prix/2021-04-17_Qualifying/


,Utc,QualifyingPart,SessionStatus,TrackStatus
0,2021-04-17T11:45:03.93Z,1.0,NaN,NaN
1,2021-04-17T12:00:00.107Z,NaN,Started,NaN
2,2021-04-17T12:05:46.658Z,NaN,NaN,Yellow
3,2021-04-17T12:05:59.83Z,NaN,Aborted,NaN
4,2021-04-17T12:05:59.989Z,NaN,NaN,Red
5,2021-04-17T12:12:44.636Z,NaN,NaN,AllClear
6,2021-04-17T12:12:52.954Z,NaN,Inactive,NaN
7,2021-04-17T12:15:00.109Z,NaN,Started,NaN
8,2021-04-17T12:27:01.114Z,NaN,Finished,NaN
9,2021-04-17T12:33:02.332Z,2.0,NaN,NaN


https://livetiming.formula1.com/static/2021/2021-04-18_Emilia_Romagna_Grand_Prix/2021-04-18_Race/


,Utc,Lap,TrackStatus,SessionStatus
0,2021-04-18T12:30:07.653Z,1.0,NaN,NaN
1,2021-04-18T12:33:33.103Z,NaN,Yellow,NaN
2,2021-04-18T12:39:36.351Z,NaN,AllClear,NaN
3,2021-04-18T13:03:15.706Z,NaN,NaN,Started
4,2021-04-18T13:04:32.261Z,NaN,Yellow,NaN
...,...,...,...,...
84,2021-04-18T15:03:24.721Z,NaN,Yellow,NaN
85,2021-04-18T15:03:32.718Z,NaN,AllClear,NaN
86,2021-04-18T15:04:32.652Z,63.0,NaN,NaN
87,2021-04-18T15:05:50.523Z,NaN,NaN,Finished
